Данный блокнот содержит все имеющиеся на текущий момент сведения по работе над датасетом первых воспоминаний 'reminiscence'.

Черновой план дальнейшей работы:
- Написать функцию апдейта, считывающую всякий раз текущее состояние датафрейма и приводящее его к необходимому состоянию внутри функций
- Преобразовать отображение чисел в возрастах к единообразному
- Правильнее считать количетсво обработанных воспоминаний так: число строк, у которых поля целиком обработаны, к общему числу строк. Для мотивации -- сделать инструмент, который бы выводил номера самых коротких записей для обработки 
- Инструмент для представления исходных данных и обработанных в верстке удобной для публикации в pdf (LaTeX), в виде статьи на Medium, в fb2, epub-форматах с заголовками и содержанием
- Сделать переводную версию
- Объединить функции для удобного обновления данных при внесении исправлении в таблицу
- Преобразование в pandas строковых значений чисел в числовые
- Инструмент перечисления номеров воспоминаний, в которых не заполнено переданное поле
- Инструмент оценки полноты датафрейма с рекомендациями работы над датафреймом
- Распределения записей по числовым параметрам:
    - Число символов
    - Число слов
    - Число воспоминаний
    - Возраст
    - Активное-пассивное
    - Доли воспоминаний с упоминанием параметра (цвет, ...)
    - Соотношение между полами
- Инструменты визуализации статистических распределений
- Использовать https://github.com/natasha/natasha и pymorphy
- pymorphy: какое время используется у глаголов, частотное распределение слов, приведенных к нормальной форме, разбивать составные воспоминания по отступ в виде одной строки или специальному символу. Отношение между частями речи. Что преобладает в коротких воспоминаниях? Анализ по парам младший/старший, мама/пара, бабушка/дедушка.
- Продумать процесс работы с ячейками перевода воспоминаний

# Считывание данных из Google Sheet

Набор данных оформлен [в виде электронной таблицы на Google Drive](https://docs.google.com/spreadsheets/d/1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI/edit?usp=sharing). Доступ к Google Sheets API осуществляется в соответствии с [документацией API для Python](https://developers.google.com/sheets/api/quickstart/python).

Импортируем данные учетной записи и преобразуем в объект Pandas:

In [1]:
import json
import os.path
import pickle       # для хранения токенов
import pandas as pd
    
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET_ID = '1KSirtO9hZSmVst--GiqsBPYk6hX-xOCI-SolgiafjcI' # id гугл-таблицы
RANGE_NAME = 'Memories' # Забираем лист целиком диапазон

In [2]:
def get_data():
    """Авторизует и забирает данные из Google SpreadSheet"""
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    
    return values



## Обработка данных в Pandas

Создаем датафрейм. Удаляем те строки, что не относятся к датасету (комментарии в конце файла)

In [64]:
def get_df():
    data = get_data()
    df = pd.DataFrame.from_records(data)
    headers = df.iloc[0]  # Названия столбцов содержатся в нулевой строке таблицы
    df = pd.DataFrame(df.values[1:], columns=headers)
    df = df.set_index('№') # Номер строки соответствует номеру воспоминания
    
    # Пока не рассматриваются переводные записи. Комментарии используются лишь как заметки
    df = df.drop(columns=['Translation', 'Translation footnote', 'Комментарий'])
    
    # Добавим столбец подсчета числа символов, из которых состоит запись.
    df ['Число символов'] = df['Воспоминание'].apply(lambda x: len(x))
    
    return df

In [65]:
df = get_df()

In [66]:
df.head()

,Имя,Name,Воспоминание,Пол,Кол-во,Возраст,Ориентир возраста,Лица,Предметы,Цвета,Звуки,"Запахи, вкус",Другие ощущения,Сущности,Пространство,Время дня,Время года,Поведение,Число символов
№,,,,,,,,,,,,,,,,,,,
1,Женя,Zhenya,"Мама поставила меня на стул, лицом к окну, шел...",-,1,4,день рождения,мама,пижама,"красный, белый",голос,-,-,снег,комната,-,зима,пассивное,205
2,-,-,Мне года полтора. Я беру маленький деревянный ...,-,1,1.5,-,-,стульчик,-,-,-,-,"солнечный свет, деревянный, квадрат","коридор, комната",день,-,"активное, пассивное",174
3,Таня,Тanya,"Оно относится к несчастному возрасту, когда ме...",ж,2,1-3,ясли,"мама, папа",игрушка,-,-,суп,"страх, одиночество",лестница,"ясли, дом",-,-,"пассивное, активное",438
4,Кира,Kira,"Помню, как бабушка меня качает, а мама смеется...",ж,1,-,-,"бабушка, мама","стол, окно, волосы",седой,смех,-,-,седые длинные волосы,дом,-,-,пассивное,164
5,Настя,Nastya,"Я лежу в кровати с мамой, позднее утро, выходн...",ж,8,<7,-,"мама, бабушка, злой мальчик, девочка,","кукла, санки, коробка, спица, стеклянный стакан",-,собственный плач,-,"боль, желание иметь брата/сестру, обида, униже...",кровь,"кровать, улица, садик, комната, медицинское уч...","позднее утро, вечер",зима,"активное, пассивное",1254


Для обзора датасета создадим функции подсчета числа пустых ячеек определенного поля, а также вывода необработанной части датасета в порядке возрастания объема записи.

In [71]:
def col_empty_cells(df, col_name:str):
    """Находит пустые и NoneType ячейки столбца с именем col_name"""
    none_cells = df[col_name][~df[col_name].notna()].index
    empty_cells = df[col_name][df[col_name]==''].index
    ids = sorted(list(set(none_cells) | set(empty_cells)))
    return ids


def percent_line():
    """Определяет долю полностью обработанных записей, не содержащие пустых строк во всех полях,
    за исключением графы Комментарий. Выводятся строки, в порядке возрастания объема воспоминания"""
    # отбираем записи, содержащие не более 1 пустой графы - обычно это графа комментарий 
    df = get_df()
    df_not_ready = df[df.apply(lambda x: x.isnull().sum(), axis='columns') != 0]
    p = 100*(1-len(df_not_ready)/len(df))
    print("Число необработанных записей {0}, это {1:.1f}% датасета.".format(len(df_not_ready), p))
    return df_not_ready.sort_values(by='Число символов')

In [73]:
percent_line()

Число необработанных записей 380, это 41.1% датасета.


,Имя,Name,Воспоминание,Пол,Кол-во,Возраст,Ориентир возраста,Лица,Предметы,Цвета,Звуки,"Запахи, вкус",Другие ощущения,Сущности,Пространство,Время дня,Время года,Поведение,Число символов
№,,,,,,,,,,,,,,,,,,,
476,Майк (финн),Mike,,м,,3-4,None,None,None,None,None,None,None,None,None,None,None,None,0
475,Саша,Sasha,Мама поет колыбельную.,м,1,2-3,,мама,None,None,None,None,None,None,None,None,None,None,22
564,Арменуи,Armenui,Мы переехали в новый дом.,ж,1,2.5,None,None,None,None,None,None,None,None,None,None,None,None,25
570,Аня,Anya,Шарики в руках и площадь,ж,1,5,None,None,None,None,None,None,None,None,None,None,None,None,25
396,Эдвард,Edward,Три года. Пальцы в розетке.,м,1,3,None,None,None,None,None,None,None,None,None,None,None,None,27
629,Мадина,Madina,Как я увидела себя в зеркале.,ж,1,4,None,None,None,None,None,None,None,None,None,None,None,None,29
468,Анна,Anna,Отец ушел с моего дня рождения.,ж,1,3-4,,папа,None,None,None,None,None,None,None,None,None,None,31
622,Вика,Vika,Прогулка с родителями по парку.,ж,1,3,None,None,None,None,None,None,None,None,None,None,None,None,31
624,Лена,Lena,Ночью мама греет чайник на кухне.,ж,1,3,,мама,None,None,None,None,None,None,None,None,None,None,33
